In [ ]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd
import time

from aprl.agents import MujocoFiniteDiffDynamicsPerformance
from aprl.envs.mujoco_costs import get_cost
from experiments.common import set_seeds, make_env, fit_ilqr, \
                               on_iteration, receding, multi_evaluate

In [4]:
seed = 42
set_seeds(seed)

env_name = 'InvertedDoublePendulum-v2'
env, us_init = make_env(env_name, seed=seed, horizon=100)
dynamics = {'performance': MujocoFiniteDiffDynamicsPerformance(env)}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

costs = {k: get_cost(env_name) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], len(us_init))  
         for k, dyn in dynamics.items()}

In [5]:
xs, us = fit_ilqr(ilqrs, x0s, us_init)

dict_keys(['performance'])
*** Fitting performance ***
iteration -1 converged 1072.8584477859247 [-0.64029087  3.3499411   0.25855345  0.65627266  7.18869424  2.83578306] [-0.7842171]
iteration 0 failed 1072.8584477859247 [-0.64029087  3.3499411   0.25855345  0.65627266  7.18869424  2.83578306] [-0.7842171]
iteration 1 failed 1072.8584477859247 [-0.64029087  3.3499411   0.25855345  0.65627266  7.18869424  2.83578306] [-0.7842171]
iteration 2 accepted 1059.941243851374 [-0.66334038  3.32649992  0.27856225  0.57285335  6.84846271  3.8956121 ] [-0.7839354]
iteration 3 failed 1059.941243851374 [-0.66334038  3.32649992  0.27856225  0.57285335  6.84846271  3.8956121 ] [-0.7839354]
iteration 4 accepted 915.3045732401388 [-0.59995386  3.0761087   0.20781475  0.04160346  3.4572538  11.10813061] [-0.73891616]
iteration 5 failed 915.3045732401388 [-0.59995386  3.0761087   0.20781475  0.04160346  3.4572538  11.10813061] [-0.73891616]
iteration 6 accepted 357.29716793786497 [ 0.05763148  2.88049626

# Receding horizon

In [9]:
for k, ilqr in ilqrs.items():
    underlying = 'receding_' + k
    dynamics[underlying] = dynamics[k]
    x0s[underlying] = x0s[k]
    xs[underlying], us[underlying] = receding(env, ilqr, x0s[k], us_init, horizon=2000, step_size=5)

iteration 0 x = [[-2.51713244e-02 -3.13228967e-03  8.41541642e-02  2.78393152e-01
  -8.96038897e-02 -8.23645605e-02]
 [-2.30546428e-02 -2.73084678e-03  8.18630811e-02  1.44947338e-01
   1.69889282e-01 -3.75888164e-01]
 [-2.20716243e-02 -1.47207303e-04  7.71597407e-02  5.16305864e-02
   3.47088796e-01 -5.65405536e-01]
 [-2.18733970e-02  3.90881604e-03  7.09333390e-02 -1.20300173e-02
   4.64519595e-01 -6.80839525e-01]
 [-2.22015715e-02  8.92125159e-03  6.38169963e-02 -5.36618124e-02
   5.38462678e-01 -7.43584246e-01]
 [-2.28637554e-02  1.45145173e-02  5.62601789e-02 -7.88392976e-02
   5.80737757e-01 -7.69035389e-01]], u = [[-0.28724512]
 [-0.20120426]
 [-0.13727081]
 [-0.08946668]
 [-0.05347044]]
iteration 5 x = [[-0.02286376  0.01451452  0.05626018 -0.0788393   0.58073776 -0.76903539]
 [-0.02374376  0.02047286  0.04850594 -0.09722919  0.61150513 -0.78311848]
 [-0.02474382  0.02661982  0.04074632 -0.10285549  0.61848128 -0.77013834]
 [-0.02575514  0.03275667  0.03320183 -0.09948176  0.60

iteration 80 x = [[ 0.36537107 -0.07867554  0.0846208   0.54366648  0.04739417 -0.13348775]
 [ 0.3704281  -0.07758204  0.08269588  0.46772806  0.17140011 -0.25172215]
 [ 0.37473421 -0.07526281  0.07960241  0.39346679  0.29266331 -0.36746151]
 [ 0.37831124 -0.07175305  0.07537199  0.3218953   0.40962317 -0.47936359]
 [ 0.38119218 -0.06710585  0.07005279  0.25423315  0.52025852 -0.58544566]
 [ 0.38342432 -0.0613991   0.06371902  0.19212376  0.62162742 -0.68247949]], u = [[-0.17499133]
 [-0.17113008]
 [-0.16488104]
 [-0.15581784]
 [-0.14305549]]
iteration 85 x = [[ 0.38342432 -0.0613991   0.06371902  0.19212376  0.62162742 -0.68247949]
 [ 0.38507664 -0.05475125  0.05649177  0.1382606   0.70856048 -0.76430584]
 [ 0.38626098 -0.04736561  0.04859664  0.09851905  0.76923829 -0.81616589]
 [ 0.3872358  -0.03974579  0.0406399   0.09635921  0.75540538 -0.77663784]
 [ 0.38694812 -0.02968644  0.02977036 -0.15402859  1.25738616 -1.399226  ]
 [ 0.38507074 -0.01647507  0.01503563 -0.22159282  1.386035

iteration 160 x = [[ 9.90415299e-01 -5.96876235e-01  9.04216109e-02  6.48747151e-03
  -1.74211235e+00  1.54072215e+00]
 [ 9.90455025e-01 -6.14871826e-01  1.06677821e-01  1.93082383e-03
  -1.85842685e+00  1.71262516e+00]
 [ 9.90449507e-01 -6.34051152e-01  1.24691141e-01 -2.32129987e-03
  -1.97920765e+00  1.89242691e+00]
 [ 9.90405654e-01 -6.54464512e-01  1.44544174e-01 -5.69287339e-03
  -2.10523921e+00  2.08040934e+00]
 [ 9.90340659e-01 -6.76175507e-01  1.66325433e-01 -6.84504494e-03
  -2.23820742e+00  2.27730357e+00]
 [ 9.88502641e-01 -6.96780008e-01  1.87865359e-01 -3.59779838e-01
  -1.88863957e+00  2.04384709e+00]], u = [[ 0.20229663]
 [ 0.11043357]
 [ 0.0173227 ]
 [-0.01222115]
 [-1.10608256]]
iteration 165 x = [[ 0.98850264 -0.69678001  0.18786536 -0.35977984 -1.88863957  2.04384709]
 [ 0.98508887 -0.71663638  0.20962113 -0.32328202 -2.08221625  2.30619063]
 [ 0.98303474 -0.7396986   0.23493795 -0.08858849 -2.52609559  2.74734778]
 [ 0.98187808 -0.76535808  0.26323965 -0.14284165 -

iteration 230 x = [[ 2.73183899e-02 -4.13950309e+00 -1.25421616e+00 -1.97913096e+00
  -7.98246547e+00  2.44503639e+00]
 [ 6.18061755e-03 -4.22126417e+00 -1.21949410e+00 -2.24551861e+00
  -8.35457813e+00  4.48969648e+00]
 [-1.39812138e-02 -4.30238413e+00 -1.17662856e+00 -1.78950280e+00
  -7.87763798e+00  4.06989805e+00]
 [-2.96597712e-02 -4.37897825e+00 -1.13832358e+00 -1.34858014e+00
  -7.44912271e+00  3.58170254e+00]
 [-4.26291262e-02 -4.45302987e+00 -1.09976177e+00 -1.24566530e+00
  -7.36046842e+00  4.12225170e+00]
 [-5.62754840e-02 -4.52759671e+00 -1.05053121e+00 -1.48228128e+00
  -7.54591022e+00  5.71491182e+00]], u = [[-7.76929808]
 [ 1.54389251]
 [ 1.04128707]
 [ 0.04881958]
 [-0.95410737]]
iteration 235 x = [[-0.05627548 -4.52759671 -1.05053121 -1.48228128 -7.54591022  5.71491182]
 [-0.07232456 -4.6039001  -0.98537643 -1.72608686 -7.71089242  7.30926702]
 [-0.09076648 -4.68176894 -0.90446066 -1.96062484 -7.86203283  8.86898519]
 [-0.10827206 -4.75925506 -0.81773823 -1.54195236 -

iteration 315 x = [[ -0.9966562   -9.02100107  -2.36995644  -0.58526363 -13.38618398
   -3.795233  ]
 [ -0.99967451  -9.1507928   -2.4010585   -0.09185734 -12.68305535
   -2.42119755]
 [ -0.99944685  -9.27593884  -2.4181154    0.10209539 -12.38811513
   -0.97386598]
 [ -0.99805983  -9.39885894  -2.42027807   0.15908724 -12.20249799
    0.55312711]
 [ -0.99637959  -9.51973687  -2.40704767   0.16798356 -11.96895592
    2.09941133]
 [ -0.99485148  -9.63812758  -2.37815025   0.13656037 -11.69380124
    3.67552597]], u = [[-0.73969706]
 [-1.85330168]
 [-1.58182641]
 [-0.21006125]
 [-0.92324335]]
iteration 320 x = [[ -0.99485148  -9.63812758  -2.37815025   0.13656037 -11.69380124
    3.67552597]
 [ -0.99082185  -9.74863284  -2.33979013   0.66913977 -10.42206827
    3.93887693]
 [ -0.98149828  -9.84693098  -2.300382     1.19323894  -9.25732617
    3.89874897]
 [ -0.9670147   -9.93418628  -2.26257792   1.70028898  -8.21385923
    3.62900147]
 [ -0.9516394  -10.01699255  -2.21835792   1.3783606

...

iteration 1685 x = [[ 0.24473736 26.7356214   5.01537301  1.44839624  6.19156638 -6.89257219]
 [ 0.26118483 26.7987947   4.94338254  1.84228286  6.4480493  -7.50214329]
 [ 0.28160835 26.86468669  4.86541042  2.24377415  6.73592096 -8.08766055]
 [ 0.3060892  26.93363344  4.78173381  2.65394222  7.060011   -8.64184491]
 [ 0.33472042 27.00603208  4.69270213  3.07409482  7.42780678 -9.15743053]
 [ 0.36760998 27.08237081  4.59874017  3.50593978  7.85017254 -9.62646622]], u = [[2.72498216]
 [1.5756091 ]
 [1.35875953]
 [1.06064884]
 [2.95972626]]
iteration 1690 x = [[  0.36760998  27.08237081   4.59874017   3.50593978   7.85017254
   -9.62646622]
 [  0.4048858   27.16326734   4.50035979   3.95179854   8.34233884
  -10.03928726]
 [  0.44336919  27.24724788   4.40723727   3.74485717   8.45875979
   -8.58574863]
 [  0.48090191  27.3333681    4.32566917   3.762675     8.77199391
   -7.72058388]
 [  0.51722228  27.42154899   4.25673248   3.50015638   8.85842964
   -6.06219836]
 [  0.55258544  27.5

iteration 1765 x = [[-1.23822239e-01  2.57530364e+01  1.20511783e+01 -3.03395322e-01
  -3.89048689e+00  7.49076211e+00]
 [-1.25231862e-01  2.57124298e+01  1.21267231e+01  2.26242941e-02
  -4.23873029e+00  7.63863117e+00]
 [-1.23602604e-01  2.56684444e+01  1.22041721e+01  3.04459450e-01
  -4.56651726e+00  7.87088676e+00]
 [-1.20723109e-01  2.56233466e+01  1.22825965e+01  2.70931811e-01
  -4.45081848e+00  7.80599532e+00]
 [-1.16494776e-01  2.55767100e+01  1.23627907e+01  5.76447586e-01
  -4.88646214e+00  8.25511677e+00]
 [-1.08556189e-01  2.55243623e+01  1.24492878e+01  1.01418924e+00
  -5.59785158e+00  9.07550438e+00]], u = [[ 0.81591656]
 [ 0.68524774]
 [-0.11037388]
 [ 0.71482646]
 [ 1.1800033 ]]
iteration 1770 x = [[-0.10855619 25.52436229 12.44928779  1.01418924 -5.59785158  9.07550438]
 [-0.09615034 25.46447232 12.54484722  1.47001465 -6.39249006 10.05709873]
 [-0.08276915 25.40356448 12.64135835  1.20284856 -5.77570681  9.21274641]
 [-0.07213997 25.3491301  12.72869225  0.92054575

iteration 1835 x = [[-0.46412869 20.82326181 16.18866795 -1.78467262 -8.27428071  5.75499059]
 [-0.48112973 20.74284437 16.24675971 -1.61836075 -7.81719276  5.86294238]
 [-0.49661071 20.6667694  16.30602253 -1.48033772 -7.40462636  5.98846716]
 [-0.51078703 20.59462262 16.36650634 -1.35714484 -7.03090027  6.10659435]
 [-0.52366049 20.52598828 16.42787463 -1.21943113 -6.70209958  6.16516836]
 [-0.53498025 20.46035981 16.48936316 -1.04598788 -6.42984023  6.13094176]], u = [[0.1343779 ]
 [0.09510121]
 [0.09292358]
 [0.17822528]
 [0.32277748]]
iteration 1840 x = [[-0.53498025 20.46035981 16.48936316 -1.04598788 -6.42984023  6.13094176]
 [-0.54419846 20.39704704 16.54975664 -0.79856002 -6.23940365  5.94735851]
 [-0.5527162  20.33656945 16.61124339 -0.90705052 -5.85812164  6.34445302]
 [-0.56044077 20.2784622  16.67338017 -0.63812172 -5.76865192  6.08390556]
 [-0.56735689 20.22254611 16.73588209 -0.74684662 -5.41676116  6.41057096]
 [-0.57589848 20.17047969 16.80221021 -0.96347435 -4.9988111

iteration 1915 x = [[-0.88222076 18.79674575 18.97634239  1.41964541 -3.01584713  2.71835478]
 [-0.87034081 18.7712121  18.9978516   0.95698924 -2.09603832  1.59637988]
 [-0.86306891 18.75473422 19.0084952   0.49791276 -1.20327424  0.54151428]
 [-0.85786644 18.74203099 19.01505015  0.54265785 -1.33790183  0.77009673]
 [-0.85437006 18.73229388 19.01861284  0.15694214 -0.61163189 -0.05262629]
 [-0.85326807 18.7268533  19.01762665  0.06353907 -0.47693391 -0.14389352]], u = [[-2.45653988]
 [-1.47552944]
 [ 0.08314611]
 [-0.85005398]
 [-0.21969058]]
iteration 1920 x = [[-0.85326807 18.7268533  19.01762665  0.06353907 -0.47693391 -0.14389352]
 [-0.85384408 18.72424275 19.01391931 -0.17866668 -0.0455153  -0.5970483 ]
 [-0.8578881  18.7280449  19.00310237 -0.63026302  0.80696878 -1.56917875]
 [-0.86646189 18.74043754 18.98241044 -1.08491958  1.67459982 -2.57694332]
 [-0.87765847 18.75767414 18.95629386 -1.15460408  1.77450631 -2.65053091]
 [-0.8887601  18.77433228 18.93137188 -1.0658151   1.55

iteration 1995 x = [[ -0.74925706  15.80186874  20.23620245  -1.57205522  -4.92742197
  -11.30830822]
 [ -0.76563505  15.75656699  20.11501616  -1.70221485  -4.10964996
  -12.97137355]
 [ -0.78263848  15.72135738  19.97452707  -1.697093    -2.90659609
  -15.17889627]
 [ -0.80170879  15.69492103  19.81448207  -2.11533532  -2.35068444
  -16.91383468]
 [ -0.82026874  15.68376289  19.62497844  -1.59496035   0.1551496
  -21.09272325]
 [ -0.83357794  15.6987733   19.39003548  -1.06628841   2.87740101
  -26.02773196]], u = [[-0.34311449]
 [-0.06911172]
 [-2.50292792]
 [ 2.1322218 ]
 [ 1.66104614]]


# Rollouts

In [8]:
multi_evaluate(env, dynamics, x0s, us)

performance
receding_performance


,rewards,lengths
performance,935.548815,100
receding_performance,1869.756658,200
